#  Scraping and Analyzing Youtube Comments in R

This script will give a detailed walktrough of how to use the Tuber package (https://cran.r-project.org/web/packages/tuber/tuber.pdf) to extract youtube comments, format them, and give some basic examples for analysis of text and contained emojis.

The script is part of an ongoing research project (https://www.researchgate.net/project/Methods-and-Tools-for-Automatic-Sampling-and-Analysis-of-YouTube-Comments) and will be subject to change. If you use substantive parts of this script as part of your own research, please cite it in the following way:

**(INSERT CITATION HERE)**

This notebook is structured in five parts:

1) [Setting up your local R environment](#Setup)

2) [Create and athenticate an account for the Youtube API](#Authentification)

3) [Extract data from youtute](#Extraction)

3) [Import Data to this Notebook to do exemplary analysis online, if you don't want a local version](#Import)

4) [Extract, process and analyse text and emojis from comment data](#Analysis)

You can thus either run an exemplary analysis in the browser without creating your own local files (simply start with step 3). Or setup our own local data collection script (Step 1 & 2). You can also download this notebook as an R script (File -> Download as -> R(.r)) or Markdown file (File -> Download as -> Makrdown(.md)) and use it locally to adapt our analysis with other video URLs.

## <a id='Setup'></a> Setting up Local Environment

First of all, we need to set up our R environment correctly so we are able to use the tuber package and all other
necessary packages. For this, open a new Script in R Studio.

In [ ]:
# removing all objects from the current global environment
rm(list=ls())

Next, we will set the correct working directory. It´s best if all files that you want to use troughout your analysis are in this folder.

In [ ]:
# choose a working directory in a GUI window
dir <- choose.dir()

In [ ]:
# changing working directory
setwd(dir)

In [ ]:
# remove directory string from global environment
rm(dir)

Next, we´re setting options so that text strings are not automatically detected as factor variables

In [ ]:
# set options parameter so that textstrings are not interpreted as factor variables
options(stringsAsFactors = FALSE) 

Then, we define a list of packages that we need to import

In [ ]:
# create list of necessary packages for data extraction and analysis
packages <- c("devtools",
              "tm",
              "quanteda",
              "tuber",
              "qdapRegex",
              "rlang",
              "purrr",
              "ggplot2",
              "syuzhet") 

We than install the packages and attach them in our R session

In [ ]:
# installing all packages in list
install.packages(packages,repos='http://cran.us.r-project.org')

In [ ]:
# attaching all packages in list
lapply(packages, library, character.only = TRUE)

There is another package that we need to install from GitHub because it is not yet on CRAN

In [ ]:
# Installing the emo package from github (not on CRAN yet)
devtools::install_github("hadley/emo")
library(emo)

Lastly, we can check if all the packages we need have been installed correctly

In [ ]:
# checking if packages have been properly attached
sessionInfo()

# <a id='Authentification'></a> Authentication for Youtube API

To get access to data from Youtube, we have to use the Youtube API (https://developers.google.com/youtube/v3/).
To do this, we need a token that identifies us when using the API so Youtube can be sure
that their Terms and Conditions are respected (e.g. there is a certain limit how much data you can access in
a given timeframe). We´ll go about this step by step:

1. If you do not have a google account that you are willing to use for this project, you have to create a new one here https://accounts.google.com/signup/v2/webcreateaccount?hl=en-GB&flowName=GlifWebSignIn&flowEntry=SignUp

2. With your google account, you have to create a "Google Project" and configure it correctly so you can get the right credentials to get access to the data with the tuber package. In case that you need help, we have set up a video showcasing and describing the process. You can find it here: https://www.youtube.com/watch?v=qLrNq0jWH84

3. Use the credentials of the project-account to authenticate your R-session. By doing this, you allow R access to all data on youtube,that you would be able to see if you went to the site logged in with this account. Be carefull to **NOT** share these credentials with anyone else you don´t want to be able to log into this account.

4. Run the authentification for the R session using the credentials from the Google project:

In [ ]:
# The credentials below are from an account that we specifically set up to scrape youtube comments.
# You have to use your own credentials when you make calls to the API. 

appID <- "" # Insert your own app Id here
appSecret <- "" # insert your own app Secret here

Upon running the following line, there will be a prompt in the console asking you to save the access token in a file. Select "No" by entering 2 in the console and hitting enter. Afterwards, a browser window should open, prompting you to log in with your google account. After logging in, you can close the browser and come back to R

In [ ]:
yt_oauth(appID,appSecret)

# <a id='Extraction'></a>Extracting Comments

First, we need the video ID of the youtube video(s) in question.
We can find it by navigating to the Video in our webbrowser, and simply
copying the last string of the URL that comes after the part "?v="

As an example, lets take the song "Baby" by Justin Bieber.
You can find it on youtube here: https://www.youtube.com/watch?v=kffacxfA7G4

So the necessary video ID would be "kffacxfA7G4"

In [ ]:
# saving the video ID in a variable
VidID <- "kffacxfA7G4"

In [ ]:
# set_results value between 20 and 100 to scrape an excerpt of the comments
Comments <- get_comment_threads(c(video_id=VidID), max_results = 100)

In [ ]:
# BEFORE your run this command, do a test run and see how quickly you can fetch a few hundred comments
# extrapolate from that whether you can safely fetch all comments at once or if you have to split
# it up into smaller chunks.

# Not recommended for the Justin Bieber Video (4.5 Million comments)

# to extract all comments
Comments <- get_all_comments(c(video_id="ENTER YOUR VIDEO ID HERE"))

**NOTE:** Your dataset might contain less comments than are displayed as total comments on YouTube. This is because the tuber package only scrapes up to five _replies_ to each comment. If a comment has more than five replies, all subsequent replies will not be extracted by the tuber package.

# <a id='Import'></a> Importing Prepared Data

Because it is not possible yet to scrape comments from this session directly, we downloaded all comments (31.08.2018) from this video: https://www.youtube.com/watch?v=DcJFdCmN98s so we can showcase the data formatting and analysis interactively in this Browser session.

**If you downloaded this as an R script or markdown and use it locally, you can ignore this section as it will not work on your local machine**

In [ ]:
# set options parameter so that textstrings are not interpreted as factor variables
options(stringsAsFactors = FALSE) 

In [ ]:
# Importing pre-downloaded Comments Datafile
load("DayumComments.Rdata")

In [ ]:
# disyplaing first 10 rows of datafile
head(Comments,10)

In [ ]:
# create list of necessary packages for data analysis
packages <- c("devtools",
              "tm",
              "quanteda",
              "tuber",
              "qdapRegex",
              "rlang",
              "purrr",
              "ggplot2",
              "syuzhet") 

In [ ]:
# attaching all packages in list
lapply(packages, library, character.only = TRUE)

In [ ]:
# Installing the emo package from github (not on CRAN yet)
devtools::install_github("hadley/emo")
library(emo)

In [ ]:
# Installing the emoGG package from github (not on CRAN yet): Displays Emoji in ggplot objects
devtools::install_github("dill/emoGG")
library(emoGG)

# <a id='Analysis'></a> Formatting the Data

To use the data effectively, we will have to format the data. Formatting will include:

- Extract Emojis from comments and format them in human-readable and R-friendly formats
- Extracting URLs from comments
- Handling special characters in comments
- Properly formatting timestamps into a format usable by R

To this end, we wrote a custom function for formatting the data. This function is still work in progress and and this point far from computationally efficient, but it might nonetheless help you to bring the comments into a format you can work with.

In [ ]:
yt_parse <- function(x){
        
        #### We need to first check first the dataframe has 15 columns (videos with replies to comments) or fewer (videos without replies to comments)
        
        if (dim(x)[2] > 13) {
                
                # only keep the relevant columns
                x <- x[,c(1,7,10,11,12,13,14)]
                
                # Convert dataframe columns to proper types
                x[,1] <- as.factor(x[,1])
                x[,2] <- as.character(x[,2])
                x[,3] <- as.numeric(x[,3])
                x[,6] <- as.character(x[,6])
                x[,7] <- as.character(x[,7])
                
                # convert timestamps into proper date-time objects
                Published <- unlist(lapply(as.character(x[,4]),function(x){paste(substr(x,1,10),substr(x,12,19),sep = "-")}))
                x[,4] <- as.POSIXct(Published, format ="%Y-%m-%d-%H:%M:%S ", tz = "UTC")
                
                Updated <- unlist(lapply(as.character(x[,5]),function(x){paste(substr(x,1,10),substr(x,12,19),sep = "-")}))
                x[,5] <- as.POSIXct(Updated, format ="%Y-%m-%d-%H:%M:%S ", tz = "UTC")
                
                
                #### Emojis
                
                ## convert Emoji names to CamelCase (if you want to learn (more) about CamelCase: https://en.wikipedia.org/wiki/Camel_case)
                simpleCap <- function(x) {
                        s <- strsplit(x, " ")[[1]]
                        paste(toupper(substring(s, 1,1)), substring(s, 2),
                              sep="", collapse=" ")
                }
                
                ## detect and replace Emojis in the comments
                
                ReplaceEM <- function(x) {
                        
                        
                        # Setup: import Emoji List
                        EmoticonList <- jis
                        
                        ListedEmojis <- as.list(jis[,4])
                        CamelCaseEmojis <- lapply(jis$name,simpleCap)
                        CollapsedEmojis <- lapply(CamelCaseEmojis,function(x){gsub(" ","",x,fixed=TRUE)})
                        EmoticonList[,4]$name <- unlist(CollapsedEmojis)
                        
                        # order the list by the length of the string to avoid partial matching of shorter strings
                        EmoticonList <- EmoticonList[rev(order(nchar(jis$emoji))),]
                        
                        # Setup: We need to assign x to a new variable so we can save the progress in the for-loop (see below)
                        New <- x
                        
                        # rm_default throws a useless warning on each iteration that we can ignore
                        oldw <- getOption("warn")
                        options(warn = -1)
                        
                        # cycle through the list and replace everything
                        # we have to add clean = FALSE and trim = FALSE to avoid deleting whitespaces that are part of the pattern
                        
                        for (i in 1:dim(EmoticonList)[1]){
                                
                                New <- rm_default(New, pattern=EmoticonList[i,3],replacement= paste0("EMOJI_", EmoticonList[i,4]$name, " "), fixed = TRUE, clean = FALSE, trim = FALSE)
                                
                        }
                        
                        # turn warning messages back on
                        options(warn = oldw)
                        
                        # output result
                        return(New)
                        
                }
                
                # Create a text column in which Emojis are replaced by their textual descriptions
                
                TextEmoRep <- ReplaceEM(x[,2])
                
                # Create a text column in which Emojis are deleted

                TextEmoDel <- emo::ji_replace_all(x[,2],"")
                
                # Create a column with only the textual descriptions of Emojis for each comment
                
                ExtractEM <- function(x){
                        
                        SpacerInsert <- gsub(" ","[{[SpAC0R]}]", x)
                        ExtractEmoji <- rm_between(SpacerInsert,"EMOJI_","[{[SpAC0R]}]",fixed=TRUE,extract = TRUE, clean= FALSE,trim=FALSE,include.markers = TRUE)
                        UnlistEmoji <- unlist(ExtractEmoji)
                        DeleteSpacer <- sapply(UnlistEmoji,function(x){gsub("[{[SpAC0R]}]"," ",x,fixed=T)})
                        names(DeleteSpacer) <- NULL
                        
                        Emoji <-paste0(DeleteSpacer,collapse="")
                        return(Emoji)
                        
                }
                
                # Extract and rename Emojis
                Emoji <- sapply(TextEmoRep,ExtractEM)
                
                #### URLs
                
                # Extract URLs from comments
                
                Links <- qdapRegex::rm_url(x[,2], extract = TRUE)
                Links <- I(Links)
                
                #### Combine everything into one dataframe

                a <- cbind.data.frame(x[,1],Emoji)
                
                df <- cbind.data.frame(x[,1],x[,2],TextEmoRep,TextEmoDel,Emoji,x[,3],Links,x[,4],x[,5],x[,6],x[,7])
                names(df) <- c("Author","Text","TextEmojiReplaced","TextEmojiDeleted","Emoji","LikeCount","URL","Published","Updated","ModerationStatus","CommentID")
                row.names(df) <- NULL
                
                
        }
        
        else {
                
                # only keep relevant columns
                x <- x[,c(1,7,10,11,12)]
                
                # Convert dataframe columns to proper types
                x[,1] <- as.factor(x[,1])
                x[,2] <- as.character(x[,2])
                x[,3] <- as.numeric(x[,3])
                
                # convert timestamps into proper date-time objects
                Published <- unlist(lapply(as.character(x[,4]),function(x){paste(substr(x,1,10),substr(x,12,19),sep = "-")}))
                x[,4] <- as.POSIXct(Published, format ="%Y-%m-%d-%H:%M:%S ", tz = "UTC")
                
                Updated <- unlist(lapply(as.character(x[,5]),function(x){paste(substr(x,1,10),substr(x,12,19),sep = "-")}))
                x[,5] <- as.POSIXct(Updated, format ="%Y-%m-%d-%H:%M:%S ", tz = "UTC")
                
                
                #### Emoji
                
                ## convert Emoji names to CamelCase (What is CamelCase? https://en.wikipedia.org/wiki/Camel_case)
                simpleCap <- function(x) {
                        s <- strsplit(x, " ")[[1]]
                        paste(toupper(substring(s, 1,1)), substring(s, 2),
                              sep="", collapse=" ")
                }
                
                ## detect and replace Emojis in the comments
                
                ReplaceEM <- function(x) {
                        
                        
                        # Setup: import Emoji List
                        EmoticonList <- jis
                        
                        ListedEmojis <- as.list(jis[,4])
                        CamelCaseEmojis <- lapply(jis$name,simpleCap)
                        CollapsedEmojis <- lapply(CamelCaseEmojis,function(x){gsub(" ","",x,fixed=TRUE)})
                        EmoticonList[,4]$name <- unlist(CollapsedEmojis)
                        
                        # order the list by the length of the string to avoid partial matching of shorter strings
                        EmoticonList <- EmoticonList[rev(order(nchar(jis$emoji))),]
                        
                        # Setup: We need to assign x to a new variable so we can save the progress in the for-loop (see below)
                        New <- x
                        
                        # rm_default throws a useless warning on each iteration that we can ignore
                        oldw <- getOption("warn")
                        options(warn = -1)
                        
                        # cycle through the list and replace everything
                        # we have to add clean = FALSE and trim = FALSE to avoid deleting whitespaces that are part of the pattern.
                        
                        for (i in 1:dim(EmoticonList)[1]){
                                
                                New <- rm_default(New, pattern=EmoticonList[i,3],replacement= paste0("EMOJI_", EmoticonList[i,4]$name, " "), fixed = TRUE, clean = FALSE, trim = FALSE)
                                
                        }
                        
                        
                        # turn warnings back on
                        options(warn = oldw)
                        
                        # output result
                        return(New)
                        
                }
                
                # create a text column in which Emojis are replaced by their textual descriptions
                
                TextEmoRep <- ReplaceEM(x[,2])
                
                # create a text column in which Emojis are deleted
                
                TextEmoDel <- emo::ji_replace_all(x[,2],"")
                
                # create a column with only the textual descriptions of Emojis for each comment
                
                ExtractEM <- function(x){
                        
                        SpacerInsert <- gsub(" ","[{[SpAC0R]}]", x)
                        ExtractEmoji <- rm_between(SpacerInsert,"EMOJI_","[{[SpAC0R]}]",fixed=TRUE,extract = TRUE, clean= FALSE,trim=FALSE,include.markers = TRUE)
                        UnlistEmoji <- unlist(ExtractEmoji)
                        DeleteSpacer <- sapply(UnlistEmoji,function(x){gsub("[{[SpAC0R]}]"," ",x,fixed=T)})
                        names(DeleteSpacer) <- NULL
                        
                        Emoji <-paste0(DeleteSpacer,collapse="")
                        return(Emoji)
                        
                }
                
                # Extract and rename Emojis
                Emoji <- sapply(TextEmoRep,ExtractEM)
                
                
                #### URLs
                
                # Extract URLs from comments
                
                Links <- qdapRegex::rm_url(x[,2], extract = TRUE)
                Links <- I(Links)
                
                #### Combine everything into one dataframe
                
                df <- cbind.data.frame(x[,1],x[,2],TextEmoRep,TextEmoDel,Emoji,x[,3],Links,x[,4],x[,5])
                names(df) <- c("Author","Text","TextEmojiReplaced","TextEmojiDeleted","Emoji","LikeCount","URL","Published","Updated")
                row.names(df) <- NULL
                
                
        }
        
        
        #### Return dataframe
        
        return(df)
        
}

We can simply use this function on the comments dataframe that we extracted to get a new dataframe that is
formatted nicely

In [ ]:
# using the function to format the "Comments" dataframe (may take a while, depending on the size of the Comment object)
FormattedComments <- yt_parse(Comments)

In [ ]:
# Displaying first 10 formatted comments
head(FormattedComments,10)

# Analysis of Text

**Disclaimer**: The textual analysis is adapted from: https://docs.quanteda.io/articles/pkgdown/examples/plotting.html

For the textual analysis, we will use the column of our formatted dataframe that does **not** contain any Emojis in any format.

In [ ]:
# Displaying first 10 elements of the column we will use for the text analysis
head(FormattedComments$TextEmojiDeleted,10)

Next, we will tokenize the the comments (e.g. splitting them into single words). At the same time, we will remove numbers, punctuation,
seperators, symbols, hyphens and URLs.

In [ ]:
## tokenizing the comments (splitting them into single words and signs)
toks <- tokens(char_tolower(FormattedComments$TextEmojiDeleted),
               remove_numbers = TRUE,
               remove_punct = TRUE,
               remove_separators = TRUE,
               remove_symbols = TRUE,
               remove_hyphens = TRUE,
               remove_url = TRUE)

In [ ]:
# displaying the first 10 tokenized comments
toks[1:10]

Next, we will create a document frequency matrix (https://en.wikipedia.org/wiki/Document-term_matrix .
In our case, a document is a comment, so  put simply, this matrix counts how often each of the words contained in any comment is appearing in every single comment. We do this while removing Stopwords (https://en.wikipedia.org/wiki/Stop_words), which can negatively influence the analysis.

In [ ]:
# We build a document frequency matrix while removing Stopwords
# Stopwords are very frequent words that occur in all texts (e.g. "a","but","it")
commentsDfm <- dfm(toks, remove = quanteda::stopwords("english"))

In [ ]:
# We can display the frequency of terms in the documents
TermFreq <- textstat_frequency(commentsDfm)
TermFreq[1:20]

### Visualizing by total occurances

In this step, we want to use the DFM we created to visualize the most freqeuent tokens across all comments. First, we order
the tokens, then we plot their frequency.

In [ ]:
# Sort by reverse frequency order
TermFreq$feature <- with(TermFreq, reorder(feature, -frequency))

In [ ]:
# Plotting the x most common tokens (you can change x to suit your needs)
x <- 15

ggplot(TermFreq[1:x], aes(x = feature, y = frequency)) +
        geom_point() + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1))

This overview might be biased because it just counts the total sum of accurances of each tokens across all comments. It might thus be that there is only one person spamming the same word hundreds of times in a single comment. To mitigate this, we will also count the number of comments that contain each token at least once.

In [ ]:
# ordering by occurance in dcuments instead of total frequency
DocFreq <- TermFreq[order(-TermFreq$docfreq),]

In [ ]:
# Plotting the x tokens that are used in the highest number of comments (you can change x to suit your needs)
x <- 15

ggplot(DocFreq[1:x], aes(x = feature, y = docfreq)) +
        geom_point() + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1))

### Custom Stopwords

After inspecting the most frequent terms, we might want to exclude certain terms that are not indicative for the comments (e.g. the word "video") or certain words that are used by spammers (e.g. "viagra"). Which words to exclude is the individual decision of each researcher. You should be carefull with designing this list for your needs and be transparent which words you excluded and why.

In [ ]:
# Custom Stopword List
CustomStops <- c("d","xd","oh","lol")

# This is just an example, you should carefully create your own list

In [ ]:
# We can create another document-frequency matrix that excludes the Custom Stopwords that we just defined, and then rerun the code above to update our results
commentsDfm <- dfm(toks, remove = c(quanteda::stopwords("english"),CustomStops))

In [ ]:
# rerunning steps from above in one cell with new DFM (excluding custom stop words)
TermFreq <- textstat_frequency(commentsDfm)

TermFreq$feature <- with(TermFreq, reorder(feature, -frequency))

x <- 25

ggplot(TermFreq[1:x], aes(x = feature, y = frequency)) +
        geom_point() + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1))

### Word Cloud

You can also use the DFM to very easily create a wordcloud to visualize the most frequent tokens

In [ ]:
# Wordcloud for most frequently used Terms
set.seed(12345)
textplot_wordcloud(dfm_select(commentsDfm, min_nchar=3),
                   random_order=FALSE,
                   max_words=100)

# Sentiment Analysis of Text

We can use Sentiment Analysis (https://en.wikipedia.org/wiki/Sentiment_analysis) on the comments to get an intuition about peoples opinions towards the content of the videos. Sentiment analysis works by comparing the tokens in each comment to a dictionary of words with an attached sentiment rating. For example, the word "fuck" would have a negative sentiment rating in the dictionary while the word "love" would have a positive sentiment rating. If we add the sentiment scores of all tokens in a given comment, we get an overall sentiment of that comment.

Of course, the results dependend on the kind of dictionary that is used. We chose the AFINN dictionary (http://www2.imm.dtu.dk/pubdb/views/publication_details.php?id=6010), because it is
based on the language in microblogs and thus might capture the slang/tone of online comments better than other dictionaries.

However, the choice of the dictionary is up to the individual researcher.

In [ ]:
# removing any punctuation and numbers from the vector from obscure Emojis that are not contained in the Emoji database,
# for example https://emojipedia.org/reversed-thumbs-up-sign/
FormattedComments$TextEmojiDeleted <- gsub("[[:punct:][:blank:]]+", " ", FormattedComments$TextEmojiDeleted)
FormattedComments$TextEmojiDeleted <- gsub('[[:digit:]]+', '', FormattedComments$TextEmojiDeleted)

In [ ]:
as.list(head(FormattedComments$TextEmojiDeleted,20))

In [ ]:
# getting sentiment scores per comment
CommentSentiment <- syuzhet::get_sentiment(FormattedComments$TextEmojiDeleted, method = "afinn")

Lets get some summary statistics and a basic vizualisation of sentiment across all comments

In [ ]:
# Basic statistics
summary(CommentSentiment)
boxplot(CommentSentiment)

We can also manually inspect comments that have extreme ratings as it´s always good to check outliers

In [ ]:
# displaying comments with a sentiment score below x
x  <- -10
as.list(FormattedComments$TextEmojiDeleted[CommentSentiment < x])

In [ ]:
# disyplaying comments with a sentiment score above x
x <- 10
as.list(FormattedComments$TextEmojiDeleted[CommentSentiment > x])

In [ ]:
# displaying most negative/positive comment
FormattedComments$TextEmojiDeleted[CommentSentiment == min(CommentSentiment)]
FormattedComments$TextEmojiDeleted[CommentSentiment == max(CommentSentiment)]

### Visualizing Comment Sentiment

To get a better overview, we can also display the total amount of positive, negative and neutral comments. To this end,
we need to create a new dataframe as a crutch to categorize the comments first.

We can also display the total distribution of sentiment in comments and overlay the mean.

In [ ]:
# building helper Frame
Desc <- CommentSentiment
Desc[Desc > 0] <- "positive"
Desc[Desc < 0] <- "negative"
Desc[Desc == 0] <- "neutral"
df <- data.frame(FormattedComments$TextEmojiDeleted,CommentSentiment,Desc)
colnames(df) <- c("Comment","Sentiment","Desc")

In [ ]:
# displaying the amount of positive, negative and neutral comments
ggplot(data=df, aes(x=Desc, fill = Desc)) +
        geom_bar(stat='count')

In [ ]:
# distribution of comment sentiments
ggplot(df, aes(x=Sentiment)) +
        geom_histogram(binwidth = 1) +
        geom_vline(aes(xintercept=mean(Sentiment)),
           color="black", linetype="dashed", size=1)

# Emoji Analysis

So far, we only analyzed the text of the comments but we can also analyze the used Emojis in the comments.
To this end, we first format NA values correctly for comments that do not contain any Emojis.
Second, we tokenize the Emojis just as we did with the text strings. Then, we build an EmojiFreqeuncy Matrix
that counts how often each Emojis is contained in every comment. Lastly, we visualize our results.

In [ ]:
# Formatting NA´s correctly
FormattedComments$Emoji[FormattedComments$Emoji == "NA"] <- NA

In [ ]:
# removing spaces at the end of the string
FormattedComments$Emoji <- substr(FormattedComments$Emoji, 1, nchar(FormattedComments$Emoji)-1)

In [ ]:
# tokinizing
EmojiToks <- tokens(FormattedComments$Emoji)

In [ ]:
# Displaying the Emojis in the first 10 comments
EmojiToks[0:10]

In [ ]:
# We build an Emoji Frequency Matrix, excluding "NA" as a term
EmojiDfm <- dfm(EmojiToks,remove = "NA")

In [ ]:
# We can display the frequency of Emojis in the documents
EmojiFreq <- textstat_frequency(EmojiDfm)
EmojiFreq

In [ ]:
# We can also get a more sparse overview of the x top Emojis in the comments
x = 20
topfeatures(EmojiDfm,x)

### Visualizing by total occurances

Using the EmojiFrequency Matrix, we can plot the most frequently occuring Emojis across all comments

In [ ]:
# Sort by reverse frequency order
EmojiFreq$feature <- with(EmojiFreq, reorder(feature, -frequency))

# Defining x
x <- 30

# Plotting x most common Emojis
ggplot(EmojiFreq[1:x], aes(x = feature, y = frequency)) +
        geom_point() + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1))

As a nice way of visulaizing the distribution, we can use the emoGG package to replace the points in the scatterplot
with the actual Emojis that they represent. For this, we have to create a mapping for each. Because this involves searching the
database manually for the emoji hex codes used by the emoGG package, we will restrict ourselves here to the 10 most commonly used Emojis.

In [ ]:
mapping1 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_facewithtearsofjoy",], aes(feature,frequency), emoji = "1f602")
mapping2 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_hamburger",], aes(feature,frequency), emoji = "1f354")
mapping3 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_frenchfries",], aes(feature,frequency), emoji = "1f35f")
mapping4 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_smilingfacewithsunglasses",], aes(feature,frequency), emoji = "1f60e")
mapping5 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_smilingface",], aes(feature,frequency), emoji = "263a")
mapping6 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_fire",], aes(feature,frequency), emoji = "1f525")
mapping7 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_loudlycryingface",], aes(feature,frequency), emoji = "1f62d")
mapping8 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_smilingfacewithheart-eyes",], aes(feature,frequency), emoji = "1f60d")
mapping9 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_rollingonthefloorlaughing",], aes(feature,frequency), emoji = "1f923")
mapping10 <- geom_emoji(data = EmojiFreq[EmojiFreq$feature == "emoji_redheart",], aes(feature,frequency), emoji = "2764")

In [ ]:
# Sort by reverse frequency order
EmojiFreq$feature <- with(EmojiFreq, reorder(feature, -frequency))

# Plotting x most common Emojis
ggplot(EmojiFreq[1:10], aes(x = feature, y = frequency)) +
        geom_point() + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
        mapping1 +
        mapping2 +
        mapping3 +
        mapping4 +
        mapping5 +
        mapping6 +
        mapping7 +
        mapping8 +
        mapping9 +
        mapping10

### Visualizing by number of comments containing the Emoji at least once

This overview might be biased because it just counts the total sum of accurances of each Emoji across all comments. It might thus be that there is only one person spamming the same Emoji hundreds of times in a single comment. To mitigate this, we will also count the number of comments that contain each Emoji at least once.

Basically, we´re counting the number of comments that do contain the Emoji.

In [ ]:
# sort by reverse document frequency order
EmojiFreq$feature <- with(EmojiFreq, reorder(feature, -docfreq))

# Plotting 30 most frequent Emoji
ggplot(EmojiFreq[1:30], aes(x = feature, y = docfreq)) +
        geom_point() + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1))

In [ ]:
# Creating a new frame order by document occurance frequenc rather than overall frequency
NewOrder <- EmojiFreq[order(-EmojiFreq$docfreq),]

Again, we can visualize this nicely by using the repective emojis instead of the normal scatterplot points.

In [ ]:
mapping1 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_facewithtearsofjoy",], aes(feature,docfreq), emoji = "1f602")
mapping2 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_hamburger",], aes(feature,docfreq), emoji = "1f354")
mapping3 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_loudlycryingface",], aes(feature,docfreq), emoji = "1f62d")
mapping4 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_fire",], aes(feature,docfreq), emoji = "1f525")
mapping5 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_redheart",], aes(feature,docfreq), emoji = "2764")
mapping6 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_heartsuit",], aes(feature,docfreq), emoji = "2665")
mapping7 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_frenchfries",], aes(feature,docfreq), emoji = "1f35f")
mapping8 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_rollingonthefloorlaughing",], aes(feature,docfreq), emoji = "1f923")
mapping9 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_thumbsup",], aes(feature,docfreq), emoji = "1f44d")
mapping10 <- geom_emoji(data = NewOrder[NewOrder$feature == "emoji_smilingfacewithheart-eyes",], aes(feature,docfreq), emoji = "1f60d")

In [ ]:
# Plotting 30 most frequent Emoji
ggplot(NewOrder[1:10], aes(x = feature, y = docfreq)) +
        geom_point() + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
        mapping1 +
        mapping2 +
        mapping3 +
        mapping4 +
        mapping5 +
        mapping6 +
        mapping7 +
        mapping8 +
        mapping9 +
        mapping10

# Sentiment Analysis for Emojis (experimental)

Just like text (and arguably even more so), Emojis are used to confer emotions and opinion. For this reason, we´re trying here for an explorative sentiment analysis using the Emojis in the comments. Just as for the sentences,we thus need a dicitionary that maps Emojis to Sentiments. Unfortunately, there so far seems to be only one sentiment dictionary for  the most commonly used 734 Emojis (http://kt.ijs.si/data/Emoji_sentiment_ranking/) . Thus, we currently do not have the possibility to check the results with different dictionaries and cannot inlcude all Emojis in this analysis.

In [ ]:
# importing emojis dictionary (We only get 734 different Emojis but thats the best data we have on Emoji Sentiment)
EmojiSentiments <- lexicon::emojis_sentiment

In [ ]:
# displaying the first ten rows of the Emoji Sentiment dictionary
EmojiSentiments[1:10,]

In [ ]:
# we have to match the sentiment scores to our codings of the emojis and create a quanteda dictionary object
EmojiNames <- paste0("emoji_",gsub(" ","",EmojiSentiments$name))
EmojiSentiment <- cbind.data.frame(EmojiNames,EmojiSentiments$sentiment,EmojiSentiments$polarity)
names(EmojiSentiment) <- c("word","sentiment","valence")
EmojiSentDict <- as.dictionary(EmojiSentiment[,1:2])

In [ ]:
# tokenizing the Emoji-only column in our formatted dataframe
EmojiToks <- tokens(tolower(FormattedComments$Emoji))

In [ ]:
# We can now replace the emojis in the dictionary with the corresponding sentiment scores
EmojiToksSent <- tokens_lookup(x = EmojiToks, dictionary = EmojiSentDict)

In [ ]:
# checking how many Emoji we can cover with sentiment scores
UnlistedEmojiToksSent <- unlist(EmojiToksSent)
names(UnlistedEmojiToksSent) <- NULL

After mapping the Emojis in our dataset to the sentiment scores in the dictionary, we can check how many Emojis we have in total in our dataset and how many of those we are getting sentiment mappings for.

In [ ]:
# total Emoji
NumberOfEmojiSentiments <- UnlistedEmojiToksSent[UnlistedEmojiToksSent!="NA"]
length(NumberOfEmojiSentiments)

In [ ]:
## sanity check

# Number of Emoji that couldn´t be replaced
length(grep("emoji_",NumberOfEmojiSentiments))

# number of Emoji that could be replaced
length(grep("0.",NumberOfEmojiSentiments))

# Percentage of Emoji that couldn´t be replaced
length(grep("emoji_",NumberOfEmojiSentiments))/length(NumberOfEmojiSentiments)

# Percentage of Emoji that could be replaced
length(grep("0.",NumberOfEmojiSentiments))/length(NumberOfEmojiSentiments)

We have to add sentiments for Emojis within the same comment to get an overall comment sentiment based on Emojis

In [ ]:
# Computing sentiment scores for comments based on Emoji

# only keeping the replaced sentiment scores for th Emoji vector
ReplacedEmojiSentScores <- tokens_select(EmojiToksSent,EmojiSentiment$sentiment,"keep")
ReplacedEmojiSentScores <- as.list(ReplacedEmojiSentScores)

# function to add sentiment scores of Emojis per comment
AddEmojiSentiments <- function(x){
        
        x <- sum(as.numeric(as.character(x)))
        return(x)
        
}


AdditiveEmojiSentiment <- lapply(ReplacedEmojiSentScores,AddEmojiSentiments)
AdditiveEmojiSentiment[AdditiveEmojiSentiment == 0] <- NA
AdditiveEmojiSentiment <- unlist(AdditiveEmojiSentiment)

We can now visualize the results

In [ ]:
# plotting histogram for distribution of Emoji Sentiment Scores
hist(AdditiveEmojiSentiment)

In [ ]:
## correlation between Emoji sentiment score and text Sentiment Score
cor(CommentSentiment,AdditiveEmojiSentiment,use="complete.obs")

In [ ]:
## plotting the relationship
plot(CommentSentiment,AdditiveEmojiSentiment)